In [14]:
# load gpt-4 model
import os
import time
import openai
import pandas as pd
import jsonlines
import numpy as np

from tqdm import tqdm



In [ ]:
# input your API key here
openai.api_key = "YOUR_API_KEY"

A sample testing

In [8]:
text = "Your text here"
prompt = """
Consider the following comment, and carefully answer the questions in each step to conclude whether it is hate speech or not: 
comment:"{text}"

Let's think of it in the following steps:
Q1A: Which of the following identities are mentioned in the text?
'race', 'nationality', 'age', 'political', 'religion', 'disability', '(anti-)masker', '(anti-)vaxxer'
Q1B: Are there any individuals mentioned explicitly by their name?
Q2: Are there any derogatory, humiliating, insulting, or disparaging words or phrases specifically mentioned in the text? (Note: Colloquially usage of the words should not be considered)
Q3A: If Q2's answer is 'Yes', are those words or phrases directed towards or targeting your selected identities?
Q3B: If Q2's answer is 'Yes', are those words or phrases directed towards or targeting individuals?
Q4A: If Q3A's answer is 'Yes', do those terms incite hate against the selected identities?
Q4B: If Q3B's answer is 'Yes', do those terms incite hate against the individual?
Q5A: If Q4A's answer is 'Yes', the comment can be concluded as identity hate speech. Tell me your final conclusion: 'Identity Hate' or 'Non-hate'.
Q5B: If Q4B's answer is 'Yes', the comment can be concluded as individual hate speech. Tell me your final conclusion: 'Individual Hate' or 'Non-hate'
""".format(text=text)
# print(prompt)   


**Note: the following code may need to be modified to fit the updated API**

In [5]:
completion = openai.ChatCompletion.create(
    model="gpt-4", 
    messages=[{"role": "user", "content": prompt}]
)
sentence  = completion.choices[0]['message']['content']

In [ ]:
print(sentence)

In [41]:
# try with gpt4 model with general prompt
def prompt_completion(text):
    try:
        # we use the user message to mimic the real conversation
        prompt = """
Do you think this comment is hateful?
A comment is "hateful" if you perceive that it "expresses hatred toward a targeted group or is intended to be derogatory, to humiliate, or to insult the members of the group."
comment:"{text}"
a. Yes b. No
""".format(text=text)   
        completion = openai.ChatCompletion.create(
            model="gpt-4", 
            messages=[{"role": "user", "content": prompt}]
        )
        sentence  = completion.choices[0]['message']['content']
        time.sleep(1)
    
    except Exception as e:
        sentence = f"error: {e}"
        time.sleep(1)

    return sentence

In [23]:
# try with gpt4 model
def prompt_completion_2(text):
    try:
        # we use the user message to mimic the real conversation
        # prompt = f"Do you think this comment is identity hate speech? comment:" + text + "Let's think it by the following steps:\n1. Which of the following identities are mentioned in the text?\n 'race', 'nationality', 'age', 'political', 'religion', ‘disability’, '(anti-)masker', '(anti-)vaxxer'\n2. Are there any derogatory, humiliating, insulting or disparaging words or phrases specifically mentioned in the text?\n3. If Q2's answer is 'Yes', are those words or phrases directed towards or targeting your selected identities?\n4. If all the answers are 'Yes', the comment should be a identity hate speech.\n5. Tell me your final choice: a. Yes b. No"
        prompt = """
Consider the following comment, and carefully answer the questions in each step to conclude whether it is hate speech or not: 
comment:"{text}"

Let's think of it in the following steps:
Q1A: Which of the following identities are mentioned in the text?
'race', 'nationality', 'age', 'political', 'religion', 'disability', '(anti-)masker', '(anti-)vaxxer'
Q1B: Are there any individuals mentioned explicitly by their name?
Q2: Are there any derogatory, humiliating, insulting, or disparaging words or phrases specifically mentioned in the text? (Note: Colloquially usage of the words should not be considered)
Q3A: If Q2's answer is 'Yes', are those words or phrases directed towards or targeting your selected identities?
Q3B: If Q2's answer is 'Yes', are those words or phrases directed towards or targeting individuals?
Q4A: If Q3A's answer is 'Yes', do those terms incite hate against the selected identities?
Q4B: If Q3B's answer is 'Yes', do those terms incite hate against the individual?
Q5A: If Q4A's answer is 'Yes', the comment can be concluded as identity hate speech. Tell me your final conclusion: 'Identity Hate' or 'Non-hate'.
Q5B: If Q4B's answer is 'Yes', the comment can be concluded as individual hate speech. Tell me your final conclusion: 'Individual Hate' or 'Non-hate'
""".format(text=text)
        completion = openai.ChatCompletion.create(
            model="gpt-4", 
            messages=[{"role": "user", "content": prompt}]
        )
        sentence  = completion.choices[0]['message']['content']
        time.sleep(1)
    
    except Exception as e:
        sentence = f"Error: {e}"
        time.sleep(30)

    return sentence

**You may need to modify the following code to fit the CSV data file**

In [15]:
# Read the data
with jsonlines.open("{the data}") as f:
    data = [line for line in f]
    print("data loaded from {}".format("{the data}"))

data loaded from data/annotation.jsonl


In [20]:
# output path
output_path = 'data/gpt4_output.jsonl'

In [25]:
# generate output
for d in tqdm(data):
    d['output'] = prompt_completion_2(d['text'])
    with jsonlines.open(output_path, mode='a') as writer:
        writer.write(d)

100%|██████████| 3990/3990 [18:32:46<00:00, 16.73s/it]   


In [ ]:
# rerun the record which has error
i = 0
for d in data:
    output = d['output']
    if output.startswith('Error:'):
        text = d['text']
        result = prompt_completion_2(text)
        d['output'] = result
        i += 1
        print("the {}th record is done:".format(i))
        print(d['output'])

In [37]:
# with jsonlines.open(output_path, mode='w') as writer:
#         writer.write(data)
for d in data:
    with jsonlines.open("data_gpt4_cot.jsonl", mode='a') as writer:
        writer.write(d)